# Análisis de series temporales
Modelos Supervisados y No Supervisados.

## Casificacion Multiclase

En el presente documento intentaremos predecir cuanto tardata una entrega determinada, en un rango de 0 a 20 dias.

Para ello relizaremos una clasificacion multiclase con diferentes algoritmos, para intentar  definir cual predice de manera mas precisa el resultado esperado.

## Bibliotecas
Vamos a cargar las biblitecas necesarias para realizar el analisis.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler

## Datos
Para empezr, cargamos los datos de los envios de Marzo de 2019.

In [2]:
cols = ['service',
        'sender_zipcode',
        'receiver_zipcode',
        'sender_state',
        'receiver_state',
        'shipment_type',
        'quantity',
        'status',
        'date_created',
        'date_sent',
        'date_visit',
        'target']

df = pd.read_csv('./shipments_BR_201903.csv', usecols=cols)

In [3]:
df.sample(5)

,sender_state,sender_zipcode,receiver_state,receiver_zipcode,shipment_type,quantity,service,status,date_created,date_sent,date_visit,target
499095,SP,18110,SP,5805,express,1,0,done,2019-03-27 00:00:00,2019-03-27 16:00:00,2019-03-28 10:44:00,1
189297,SP,2013,SP,12442,express,1,0,done,2019-03-05 00:00:00,2019-03-07 15:25:00,2019-03-08 16:28:00,0
955422,RS,95800,MG,31070,standard,3,1,done,2019-03-03 00:00:00,2019-03-04 14:29:00,2019-03-13 14:25:00,6
524463,SP,9810,PR,84031,standard,1,6,done,2019-02-26 00:00:00,2019-02-27 14:18:23,2019-03-01 14:16:43,2
722945,SP,16304,PI,64049,standard,2,2,done,2019-03-03 00:00:00,2019-03-07 16:45:17,2019-03-22 13:33:00,10


Como modelo base, utilizaremos un pipeline de sklearn, que nos permita normalizar los datos y luego pasarlos a una Regresion Logistica Multiclase.

In [4]:
model = Pipeline([
    ('normalizer', MinMaxScaler()),
    ('classifier', LogisticRegression(solver='lbfgs', 
                                      multi_class='multinomial',
                                      max_iter=500)),
])

Definimos las columnas que usaremos como features para el modelo, y cual sera la columna target.

In [5]:
features = ['sender_zipcode', 'receiver_zipcode', 'service', 'shipment_type_express', 'shipment_type_standard', 'shipment_type_super']
target = 'target'

In [6]:
df = pd.get_dummies(df, columns=['shipment_type'])

Definimos una fecha de corte para separar los datos de entrenamiento y test. Para asegurar que los datos no sean modificados, trabajamos sobre una copia del dataset original

In [7]:
copy = df.copy()

cut_off = '2019-03-20'
df_train = df.query(f'date_visit <= "{cut_off}"')
df_test = df.query(f'date_created > "{cut_off}"')

X_train = df_train[features].values.astype(np.float)
y_train = df_train[target].values

X_test = df_test[features].values.astype(np.float)
y_test = df_test[target].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((673645, 6), (673645,), (76378, 6), (76378,))

In [8]:
y_train = df_train[target].values
y_test = df_test[target].values
y_train.shape, y_test.shape

((673645,), (76378,))

Entrenamos nuestro modelo con los datos de entrenamiento y test definidos.

In [9]:
%%time
result = model.fit(X_train, y_train)
result

Wall time: 17min 44s


C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Tomamos los scores del resultado para intentar definir que tan acertado fue el modelo.

In [11]:
y_pred = model.predict(X_test)

metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred, average='macro'),
    'recall': recall_score(y_test, y_pred, average='macro'),
    'f1_score': f1_score(y_test, y_pred, average='macro'),
}

metrics

C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\femancus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'tru

{'accuracy': 0.4351645761868601,
 'precision': 0.13938371709600178,
 'recall': 0.12764325966412013,
 'f1_score': 0.10468013395301896}

Como pensabamos, las métricas obtenidas son muy bajas. Además tenemos algunos warnings que nos avisa que algunas de las clases no tienen ejemplos, por lo tanto la métrica se define en 0 para esa clase, afectando aún más a la métrica promedio (que es la que finalmente estamos imprimiendo).